In [1]:
# !pip install -r ../requirements.txt

In [2]:
from datasets import load_dataset, Dataset
import pandas as pd

text_df = pd.read_csv("gigaam_inference.tsv", sep="\t").drop("EDITING_INSTRUCTION_RU", axis=1)
raw_ds = load_dataset("arood0/mmm_project_with_audio_ru_final", split="train")
ds = raw_ds.to_pandas()


/home/kmgabidullin/voice-image-editor/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
result_df = Dataset.from_pandas(ds.merge(text_df, on="IMAGE_ID", how="inner"))
result_df

Dataset({
    features: ['IMAGE_ID', 'EDITING_TYPE', 'CORE', 'MASK', 'EDITING_INSTRUCTION', 'OUTPUT_DESCRIPTION', 'INPUT_CAPTION_BY_LLAMA', 'OUTPUT_CAPTION_BY_LLAMA', 'INPUT_IMG', 'MASK_IMG', 'OUTPUT_IMG', 'EDITING_INSTRUCTION_RU', 'audio', 'hypothesis_raw'],
    num_rows: 1000
})

In [4]:
import sys
import os

# Добавляем в sys.path корень проекта: .. относительно e2e_tests
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

from src.models.image_editor import DiffusionImageEditor

In [5]:
editor = DiffusionImageEditor(
    model_type="instruct_pix2pix",
    model_name="timbrooks/instruct-pix2pix",
    device="cuda",
    num_inference_steps=10,
    strength=0.75,
    image_guidance_scale=1.5,
    guidance_scale=7.5,
    max_side=1024,
)

Инициализация DiffusionImageEditor


Loading pipeline components...: 100%|██████████| 6/6 [00:03<00:00,  1.84it/s]


Модель загружена: timbrooks/instruct-pix2pix


In [6]:
from PIL import Image
from io import BytesIO
from datasets import Dataset, Image as HFImage

image_feature = HFImage()

rows = []
sample_cnt = 300
for el in result_df:
    img = editor.edit(image=Image.open(BytesIO(el["INPUT_IMG"]["bytes"])), instruction=el["hypothesis_raw"])
    rows.append({"IMAGE_ID": el["IMAGE_ID"], "result_image": image_feature.encode_example(img)})
    sample_cnt -= 1
    if sample_cnt <= 0:
        break

100%|██████████| 10/10 [00:05<00:00,  1.99it/s]
Token indices sequence length is longer than the specified maximum sequence length for this model (79 > 77). Running this sequence through the model will result in indexing errors
The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['а .']
100%|██████████| 10/10 [00:04<00:00,  2.41it/s]
The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['�ок .']
100%|██████████| 10/10 [00:02<00:00,  3.54it/s]
The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['� справа .']
100%|██████████| 10/10 [00:01<00:00,  6.92it/s]
The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['ета пончиков .']
100%|██████████| 10/10 [00:00<00:00, 11.45it/s]


In [7]:
total_ds = Dataset.from_pandas(pd.DataFrame(rows), preserve_index=False)
total_ds = total_ds.cast_column("result_image", image_feature)

In [8]:
total_ds.push_to_hub("gab1k/mmm_project_with_ru_intermdata")

Creating parquet from Arrow format: 100%|██████████| 3/3 [00:01<00:00,  1.56ba/s]
Uploading files as a binary IO buffer is not supported by Xet Storage. Falling back to HTTP upload.
Uploading the dataset shards: 100%|██████████| 1/1 [00:19<00:00, 19.09s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/gab1k/mmm_project_with_ru_intermdata/commit/a31acbd17abe0a0c69bb2b299b14d13a51f20014', commit_message='Upload dataset', commit_description='', oid='a31acbd17abe0a0c69bb2b299b14d13a51f20014', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/gab1k/mmm_project_with_ru_intermdata', endpoint='https://huggingface.co', repo_type='dataset', repo_id='gab1k/mmm_project_with_ru_intermdata'), pr_revision=None, pr_num=None)